In [2]:
import numpy as np
import pandas as pd


In [8]:
game_data = pd.read_csv('Game_Data2.csv')
game_data.head()

,Year,Week,HomeTeam,AwayTeam,HomeScore,AwayScore,PointDiff,HWHL,Covered,HomeLine,...,HOME_DefRushDVOA,HOME_STDVOA,VISIT_TotalDVOA,VISIT_OffenseDVOA,VISIT_OffPassDVOA,VISIT_OffRushDVOA,VISIT_DefenseDVOA,VISIT_DefPassDVOA,VISIT_DefRushDVOA,VISIT_STDVOA
0,2019,6,GB,DET,23,22,1,0,0,-6.0,...,-0.221,2.000,-0.013,-0.246,0.026,-0.375,-0.014,-0.057,0.049,0.219
1,2019,6,SD,PIT,17,24,-7,1,0,-5.5,...,-0.069,-0.062,0.025,-0.056,0.161,-0.163,-0.050,0.069,-0.399,0.031
2,2019,6,DEN,TEN,16,0,16,0,1,-2.5,...,-0.628,0.085,-0.577,-0.724,-0.562,-0.692,-0.194,-0.179,-0.210,-0.047
3,2019,6,NYJ,DAL,24,22,2,0,1,8.5,...,-0.158,0.080,0.048,0.289,0.486,0.077,0.276,0.467,-0.086,0.036
4,2019,6,ARI,ATL,34,33,1,0,1,2.0,...,0.033,-0.023,-0.026,0.326,0.662,-0.034,0.337,0.802,-0.243,-0.015


In [9]:
X = game_data[['HomeLine', 'PredictTotal', 'HOME_OffPassDVOA', 'HOME_OffRushDVOA', 'HOME_DefPassDVOA', 'HOME_DefRushDVOA', 'HOME_STDVOA', 'VISIT_OffPassDVOA', 'VISIT_OffRushDVOA', 'VISIT_DefPassDVOA', 'VISIT_DefRushDVOA', 'VISIT_STDVOA' ]]
y = game_data['PointDiff']
print(X.shape, y.shape)


(1030, 12) (1030,)


In [ ]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()
model.fit(X, y)
score = model.score(X, y)
print(f"R2 Score: {score}")

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
model = LinearRegression()

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=29)
model.fit(X_train, y_train)


MSE = mean_squared_error(y_train, model.predict(X_train))
r2 = model.score(X_train, y_train)
print(f"MSE: {MSE}, R2: {r2}")

In [ ]:
import matplotlib.pyplot as plt
    
predictions = model.predict(X_test)

plt.scatter(model.predict(X_train), model.predict(X_train) - y_train, c="blue", label="Training Data")
plt.scatter(model.predict(X_test), model.predict(X_test) - y_test, c="orange", label="Testing Data")
plt.legend()
plt.hlines(y=0, xmin=y_test.min(), xmax=y_test.max())
plt.title("Residual Plot")
plt.show()

In [ ]:
from sklearn.externals import joblib
model=joblib.load('John_Madden.joblib') 

In [10]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV
model2 = LinearRegression()
parameters = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
grid2 = GridSearchCV(model2,parameters, verbose=3, cv=3, scoring='r2')


In [11]:
grid2.fit(X, y)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.7569272345453648, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.75549319562969, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.7839522114543782, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=False ................
[CV]  copy_X=True, fit_intercept=True, normalize=False, score=0.7569272345453651, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=False ................
[CV]  copy_X=True, fit_intercept=True, normalize=False, score=0.755493195629693, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=False ................
[CV]  copy_X=True, fit_intercept=True, n

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s



[CV] copy_X=True, fit_intercept=False, normalize=True ................
[CV]  copy_X=True, fit_intercept=False, normalize=True, score=0.7828456664539601, total=   0.0s
[CV] copy_X=True, fit_intercept=False, normalize=False ...............
[CV]  copy_X=True, fit_intercept=False, normalize=False, score=0.7563474873900766, total=   0.0s
[CV] copy_X=True, fit_intercept=False, normalize=False ...............
[CV]  copy_X=True, fit_intercept=False, normalize=False, score=0.759977803629363, total=   0.0s
[CV] copy_X=True, fit_intercept=False, normalize=False ...............
[CV]  copy_X=True, fit_intercept=False, normalize=False, score=0.7828456664539601, total=   0.0s
[CV] copy_X=False, fit_intercept=True, normalize=True ................
[CV]  copy_X=False, fit_intercept=True, normalize=True, score=0.7569272345453648, total=   0.0s
[CV] copy_X=False, fit_intercept=True, normalize=True ................
[CV]  copy_X=False, fit_intercept=True, normalize=True, score=0.75549319562969, total=   0.

[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed:    0.2s finished


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'fit_intercept': [True, False], 'normalize': [True, False], 'copy_X': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='r2', verbose=3)

In [16]:
print ("r2 / variance : ", grid2.best_score_)
print("Residual sum of squares: %.2f"
              % np.mean((grid2.predict(X) -y) ** 2))

r2 / variance :  0.7663805688356954
Residual sum of squares: 43.51


In [27]:
model3 = LinearRegression(fit_intercept=False, normalize=True, copy_X=True)


['John_Madden.joblib']

In [22]:
from sklearn.externals import joblib
joblib.dump(grid2, 'Pro_Set.joblib') 

['Pro_Set.joblib']

In [17]:
print(grid2.best_params_)
print(grid2.best_score_)

{'copy_X': True, 'fit_intercept': False, 'normalize': True}
0.7663805688356954


In [28]:
model3.fit(X,y)
from sklearn.externals import joblib
joblib.dump(model3,'John_Madden.joblib')

['John_Madden.joblib']

In [37]:
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV
model2 = SVC(kernel='linear')
param_grid = {'C': [ 0.001, 0.01, 0.1, 1],
              'gamma': [ 0.00001,0.0001, 0.001, 0.01, 0.1]}
grid = GridSearchCV(model2, param_grid, verbose=3, cv=3, scoring = 'r2')

In [38]:
grid.fit(X,y)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] C=0.001, gamma=1e-05 ............................................


C:\Users\nicho\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  C=0.001, gamma=1e-05, score=-0.0034550648799327455, total=   0.1s
[CV] C=0.001, gamma=1e-05 ............................................
[CV]  C=0.001, gamma=1e-05, score=-0.010042689473768407, total=   0.0s
[CV] C=0.001, gamma=1e-05 ............................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.3s remaining:    0.0s


[CV]  C=0.001, gamma=1e-05, score=-0.005949712261470008, total=   0.1s
[CV] C=0.001, gamma=0.0001 ...........................................
[CV]  C=0.001, gamma=0.0001, score=-0.0034550648799327455, total=   0.1s
[CV] C=0.001, gamma=0.0001 ...........................................
[CV]  C=0.001, gamma=0.0001, score=-0.010042689473768407, total=   0.1s
[CV] C=0.001, gamma=0.0001 ...........................................
[CV]  C=0.001, gamma=0.0001, score=-0.005949712261470008, total=   0.1s
[CV] C=0.001, gamma=0.001 ............................................
[CV]  C=0.001, gamma=0.001, score=-0.0034550648799327455, total=   0.0s
[CV] C=0.001, gamma=0.001 ............................................
[CV]  C=0.001, gamma=0.001, score=-0.010042689473768407, total=   0.1s
[CV] C=0.001, gamma=0.001 ............................................
[CV]  C=0.001, gamma=0.001, score=-0.005949712261470008, total=   0.1s
[CV] C=0.001, gamma=0.01 .............................................
[

[Parallel(n_jobs=1)]: Done  60 out of  60 | elapsed:   32.3s finished
C:\Users\nicho\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=3, error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': [0.001, 0.01, 0.1, 1], 'gamma': [1e-05, 0.0001, 0.001, 0.01, 0.1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='r2', verbose=3)

In [39]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 1, 'gamma': 1e-05}
0.5788572774224352


In [40]:
from sklearn.externals import joblib
joblib.dump(grid, 'Pro_Set.joblib') 

['Pro_Set.joblib']